In [1]:
# from ratmoseq_extract.proc import get_flips
# import numpy as np

# moseq2-dev env

In [3]:
get_flips??

Signature: get_flips(frames, flip_file=None, smoothing=None)
Source:   
def get_flips(frames, flip_file=None, smoothing=None):
    """
    Predict frames where mouse orientation is flipped to later correct.

    Args:
    frames (numpy.ndarray): frames x rows x columns, cropped mouse
    flip_file (str): path to pre-trained scipy random forest classifier
    smoothing (int): kernel size for median filter smoothing of random forest probabilities

    Returns:
    flips (numpy.array):  array for flips
    """

    try:
        clf = joblib.load(flip_file)
    except IOError:
        print(f"Could not open file {flip_file}")
        raise

    flip_class = np.where(clf.classes_ == 1)[0]
    frames = min_max_scale(frames)

    try:
        probas = clf.predict_proba(
            frames.reshape((-1, frames.shape[1] * frames.shape[2]))
        )
    except ValueError:
        if (
            hasattr(clf, "n_features_")
            and int(np.sqrt(clf.n_features_)) != frames.shape[-1]
      

In [2]:
import numpy as np
import joblib

In [3]:
frames = np.ones((10, 256, 256))

In [4]:
flip_file = '/n/groups/datta/jlove/data/rat_seq/rat_seq_paper/flip-9week.p'

In [5]:
clf = joblib.load(flip_file)

/home/jal5475/.miniconda/envs/moseq2-dev/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [7]:
type(clf)

sklearn.pipeline.Pipeline

In [12]:
import json

# Extract configuration
pipeline_config = []
for name, step in clf.steps:
    step_config = {
        "name": name,
        "class": type(step).__name__,
        "module": step.__module__,
        "params": step.get_params()  # Get the parameters of the step
    }
    pipeline_config.append(step_config)

# # Save the configuration to a JSON file
# with open('pipeline_config.json', 'w') as f:
#     json.dump(pipeline_config, f, indent=4)


In [22]:
pca = pipeline_config[0]['params']['kw_args']['pca']

In [24]:
joblib.dump(pca, '9week-pca.p')

['9week-pca.p']

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.svm import SVC

In [27]:
pipeline = Pipeline([
    ('pca_transform', FunctionTransformer(func=pca.transform, validate=False)),
    ('standardscaler', StandardScaler()),
    ('svc', SVC(probability=True))
])

In [28]:
joblib.dump(pipeline, '9week-sklearn-native.p')

['9week-sklearn-native.p']

# rat moseq lib

In [1]:
from ratmoseq_extract.proc import get_flips

In [2]:
import joblib

In [3]:
clf = joblib.load('9week-sklearn-native.p')

ModuleNotFoundError: No module named 'sklearn.decomposition.pca'

In [4]:
# no mas - this is the end of the line for now